In [ ]:
from google.colab import drive

PATH_DRIVE = '/content/drive'
drive.mount(PATH_DRIVE)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

from datetime import datetime
from pathlib import Path

PATH_BASE = Path(PATH_DRIVE)/'MyDrive'
print(os.listdir(PATH_BASE))

['Colab Notebooks', 'data', 'Projects', 'models']


In [ ]:
! pip3 install -U datasets transformers evaluate sentencepiece

import torch
import datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from evaluate import load

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
now = datetime.now()
timestamp = datetime.timestamp(now)

NAME_MODEL = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
NAME_DIR = f"mednli_{NAME_MODEL.split('/')[-1]}_{int(timestamp)}"

PATH_MODELS = PATH_BASE/'models'/NAME_DIR

dataset = datasets.load_dataset(
    'bigbio/mednli', 
    'mednli_source',
    data_dir=PATH_BASE/'data'
)
model = AutoModelForSequenceClassification.from_pretrained(NAME_MODEL, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(NAME_MODEL)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset mednli downloaded and prepared to /root/.cache/huggingface/datasets/bigbio___mednli/mednli_source-61f28563cb5c4805/1.0.0/f18e31046c4bf68bbdc505fbf3dab924a4ea60082a7f41346e3a84b54f453535. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

In [ ]:
def label_encode(data):
    labels = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
    key = data['gold_label']
    data = {'label': labels[key]}

    return data


def tokenize(data):
    data = tokenizer(
        data['sentence1'],
        data['sentence2'],
        max_length=model.config.max_position_embeddings,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    return data


def compute_metrics(y):
    logits, labels = y
    preds = logits.argmax(axis=1)
    metrics = 'precision', 'recall', 'f1'
    result = {}

    result.update(
        load('accuracy').compute(
            predictions=preds,
            references=labels
        )
    )

    for metric in metrics:
        result.update(
            load(metric).compute(
                predictions=preds,
                references=labels, 
                average='macro'
            )
        )

    return result


In [ ]:
cols = [
    'input_ids', 
    'token_type_ids',
    'attention_mask', 
    'label'
]
cols = [col for col in dataset.column_names['test'] if col not in cols]

dataset = dataset.map(label_encode)
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.remove_columns(cols)
dataset = dataset.shuffle(seed=23)

  0%|          | 0/11232 [00:00<?, ?ex/s]

  0%|          | 0/1422 [00:00<?, ?ex/s]

  0%|          | 0/1395 [00:00<?, ?ex/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
numEpoch = 5
numBatch = 16
numStep = dataset.num_rows['train'] * numEpoch / numBatch

args = TrainingArguments(
    run_name=NAME_DIR,
    num_train_epochs=numEpoch,
    per_device_train_batch_size=numBatch,
    per_device_eval_batch_size=numBatch*4,
    gradient_accumulation_steps=2,
    # eval_accumulation_steps=2,
    learning_rate=3e-5,
    weight_decay=1e-1,
    warmup_ratio=1e-2,
    warmup_steps=int(numStep/20),
    save_total_limit=3,
    save_steps=100,
    logging_steps=50,
    save_strategy='steps',
    evaluation_strategy='steps',
    output_dir=PATH_MODELS/'weights',
    logging_dir=PATH_MODELS/'logs',
    seed=23,
    data_seed=23,
    metric_for_best_model='f1',
    # auto_find_batch_size=True,
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

using `logging_steps` to initialize `eval_steps` to 50
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Both warmup_ratio and warmup_steps given, warmup_steps will override any effect of warmup_ratio during training


In [ ]:
if not os.path.exists(PATH_MODELS):
    os.mkdir(PATH_MODELS)

with open(PATH_MODELS/'args.json', 'w', encoding='utf-8') as f:
    text = args.to_json_string()
    f.write(text)

trainer.train()
trainer.save_model(PATH_MODELS/'weights')

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11232
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1755
  Number of trainable parameters = 109484547


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.105900,0.993065,0.531900,0.625361,0.531900,0.499428
100,0.821700,0.676117,0.744086,0.748108,0.744086,0.745158
150,0.667100,0.581102,0.772760,0.794734,0.772760,0.775010
200,0.587600,0.532672,0.792115,0.795858,0.792115,0.791393
250,0.533400,0.472765,0.815771,0.819375,0.815771,0.816417
300,0.568000,0.473444,0.812903,0.820018,0.812903,0.814530
350,0.508100,0.479924,0.825806,0.826114,0.825806,0.825393
400,0.406800,0.452922,0.823656,0.826094,0.823656,0.824470
450,0.386100,0.446263,0.840860,0.844181,0.840860,0.841741
500,0.435300,0.448324,0.830824,0.835377,0.830824,0.832200


***** Running Evaluation *****
  Num examples = 1395
  Batch size = 64


***** Running Evaluation *****
  Num examples = 1395
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/models/mednli_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_1675523667/weights/checkpoint-100
Configuration saved in /content/drive/MyDrive/models/mednli_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_1675523667/weights/checkpoint-100/config.json
Model weights saved in /content/drive/MyDrive/models/mednli_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_1675523667/weights/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/models/mednli_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_1675523667/weights/checkpoint-200
Configuration saved in /content/drive/MyDrive/models/mednli_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_1675523667/weights/checkpoint-200/config.json
Model w

In [ ]:
result = trainer.evaluate(dataset['test'])
df = pd.DataFrame.from_dict([result])

df.to_csv(PATH_MODELS/'result.csv', index=False, encoding='utf-8')
df

***** Running Evaluation *****
  Num examples = 1422
  Batch size = 64


,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.638928,0.840366,0.840696,0.840366,0.840385,60.2327,23.608,0.382,5.0
